
# 🥇 ETL GOLD - Generación de KPIs para análisis y Tableau

En esta etapa generamos la **capa Gold**, que consolida los datos limpios 
de la capa Silver en indicadores analíticos (KPIs) listos para consulta 
desde **AWS Athena** y visualización en **Tableau**.


In [1]:
#Importo las librerias
import pandas as pd
import boto3
from io import BytesIO
import pyarrow as pa
import pyarrow.parquet as pq


In [2]:

# ============================================================
# ⚙️ 1. Configuración de conexión a S3
# ============================================================

s3 = boto3.client('s3')
bucket_name = 'notebooks-datalake-oalvarez'

silver_key = 'silver/notebooks_ventas_clean.csv'
gold_path = 'gold/'


In [3]:

# ============================================================
# 📥 2. Lectura de datos desde la capa Silver
# ============================================================

obj = s3.get_object(Bucket=bucket_name, Key=silver_key)
df = pd.read_csv(obj['Body'])

print('✅ Dataset cargado desde Silver')
print(df.head())


✅ Dataset cargado desde Silver
    año trimestre    marca  unidades_vendidas  ingreso_total continente
0  2022        Q3       Hp            2838341   2.318925e+09       Asia
1  2023        Q3  Samsung            5201221   5.388465e+09     África
2  2023        Q3   Lenovo            2771890   2.123268e+09    Oceanía
3  2023        Q2   Lenovo            2585840   3.550358e+09     Europa
4  2022        Q1       Hp            4377789   6.028215e+09     Europa


In [4]:

# ============================================================
# 🧮 3. Generación de tablas Gold (KPIs)
# ============================================================

# --- 3.1 Market share por marca ---
market_share = (
    df.groupby('marca')['unidades_vendidas']
    .sum()
    .reset_index()
    .sort_values(by='unidades_vendidas', ascending=False)
)
market_share['market_share_%'] = (
    100 * market_share['unidades_vendidas'] / market_share['unidades_vendidas'].sum()
)

# --- 3.2 Tendencia anual top 5 marcas ---
top5_marcas = market_share['marca'].head(5)
tendencia_top5 = df[df['marca'].isin(top5_marcas)].groupby(['año', 'marca'])['unidades_vendidas'].sum().reset_index()

# --- 3.3 Unidades por continente y marca ---
unidades_continente_marca = df.groupby(['continente', 'marca'])['unidades_vendidas'].sum().reset_index()

# --- 3.4 Ingreso vs unidades + precio promedio ---
df['precio_promedio'] = df['ingreso_total'] / df['unidades_vendidas']
ingreso_vs_unidades = df[['marca', 'año', 'continente', 'unidades_vendidas', 'ingreso_total', 'precio_promedio']]

# --- 3.5 Top 5 marcas por ingreso total ---
top5_ingreso = (
    df.groupby('marca')['ingreso_total']
    .sum()
    .reset_index()
    .sort_values(by='ingreso_total', ascending=False)
    .head(5)
)


In [5]:

# ============================================================
# 💾 4. Exportar KPIs a capa Gold (S3 en formato Parquet)
# ============================================================

def upload_parquet_to_s3(df, key_folder):
    """
    Sube un DataFrame a S3 en formato Parquet de forma idempotente.
    Si el archivo ya existe, lo sobrescribe sin duplicar rutas.
    """
    buffer = BytesIO()
    table = pa.Table.from_pandas(df)
    pq.write_table(table, buffer)

    key_name = f'{gold_path}{key_folder}/data.parquet'
    s3.put_object(Bucket=bucket_name, Key=key_name, Body=buffer.getvalue())
    print(f'✅ Archivo actualizado en S3 → {key_name}')


upload_parquet_to_s3(market_share, 'market_share_por_marca')
upload_parquet_to_s3(tendencia_top5, 'tendencia_anual_top5')
upload_parquet_to_s3(unidades_continente_marca, 'unidades_por_continente_y_marca')
upload_parquet_to_s3(ingreso_vs_unidades, 'ingreso_vs_unidades')
upload_parquet_to_s3(top5_ingreso, 'top5_marcas_ingreso')



✅ Archivo actualizado en S3 → gold/market_share_por_marca/data.parquet
✅ Archivo actualizado en S3 → gold/tendencia_anual_top5/data.parquet
✅ Archivo actualizado en S3 → gold/unidades_por_continente_y_marca/data.parquet
✅ Archivo actualizado en S3 → gold/ingreso_vs_unidades/data.parquet
✅ Archivo actualizado en S3 → gold/top5_marcas_ingreso/data.parquet


In [7]:

# ============================================================
# 📊 5. Verificación rápida
# ============================================================

print("\nMarket Share:")
print(market_share.head())

print("\nTendencia Top 5:")
print(tendencia_top5.head())

print("\nUnidades por Continente y Marca:")
print(unidades_continente_marca.head())

print("\nTop 5 Ingreso Total:")
print(top5_ingreso.head())

print("\n✅ Todos los KPIs fueron generados y guardados en la capa Gold de S3.")



Market Share:
     marca  unidades_vendidas  market_share_%
5   Huawei          110825023       13.357364
2     Asus          101653189       12.251914
8      Msi           93113386       11.222641
4       Hp           85511118       10.306365
9  Samsung           84990764       10.243649

Tendencia Top 5:
    año    marca  unidades_vendidas
0  2020     Asus           21595881
1  2020       Hp           12102756
2  2020   Huawei           16536250
3  2020      Msi           15350471
4  2020  Samsung           16241685

Unidades por Continente y Marca:
  continente  marca  unidades_vendidas
0    América   Acer           11770239
1    América  Apple            6425310
2    América   Asus           22631248
3    América   Dell           13908511
4    América     Hp           17742758

Top 5 Ingreso Total:
     marca  ingreso_total
5   Huawei   1.134949e+11
8      Msi   9.804810e+10
2     Asus   9.006859e+10
9  Samsung   8.398019e+10
4       Hp   7.993633e+10

✅ Todos los KPIs fueron gene

In [11]:
# Descargar los archivos para usar de forma local (Tableau Public no permite conectar AWS)
import pandas as pd
import boto3
import pyarrow.parquet as pq
import io
import os

# ============================================================
# ⚙️ Configuración de conexión a S3
# ============================================================
s3 = boto3.client('s3')
bucket_name = 'notebooks-datalake-oalvarez'
gold_prefix = 'gold/'

# ============================================================
# 📁 Crear carpeta local "Output" si no existe
# ============================================================
output_dir = 'Output'
os.makedirs(output_dir, exist_ok=True)

# ============================================================
# 📦 Carpetas dentro de Gold (cada una tiene un data.parquet)
# ============================================================
folders = [
    'market_share_por_marca',
    'tendencia_anual_top5',
    'unidades_por_continente_y_marca',
    'ingreso_vs_unidades',
    'top5_marcas_ingreso'
]

# ============================================================
# ⬇️ Descargar desde S3 y exportar como CSV local
# ============================================================
for folder in folders:
    key = f'{gold_prefix}{folder}/data.parquet'
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    buffer = io.BytesIO(obj['Body'].read())
    table = pq.read_table(buffer)
    df = table.to_pandas()

    csv_path = os.path.join(output_dir, f'{folder}.csv')
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f'✅ Archivo exportado: {csv_path}')

print("\n🎯 Todos los archivos CSV fueron guardados correctamente en la carpeta 'Output/'")



✅ Archivo exportado: Output\market_share_por_marca.csv
✅ Archivo exportado: Output\tendencia_anual_top5.csv
✅ Archivo exportado: Output\unidades_por_continente_y_marca.csv
✅ Archivo exportado: Output\ingreso_vs_unidades.csv
✅ Archivo exportado: Output\top5_marcas_ingreso.csv

🎯 Todos los archivos CSV fueron guardados correctamente en la carpeta 'Output/'
